In [6]:
import re
import os, os.path
from os.path import splitext,isfile,join
from os import listdir
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import pandas as pd
import tqdm



In [2]:
destinationFolder = "/Users/vt2113/Desktop/DICOM #2/DynamicOutput"
inputFolder = '/Users/vt2113/Desktop/DICOM #2/Dynamic Data#2'
AVIFolder = '/Users/vt2113/Desktop/DICOM #2/DynamicAVI'


In [3]:
def mask(output):

    dim1 = output.shape[0]
    dim2 = output.shape[1]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dim2), np.arange(dim1))
    

    mask = ((m1+m2)>(int(dim2/2) +int(dim2/12))) 
    # mask = m2> dim1/dim2*(-m1+int(dim2/2)) +int(dim2/10)
    mask *=  ((m1-m2)<(int(dim2/2) -int(dim2/12)))
    # mask *= -m2< dim1/dim2*(-m1+int(dim2/2)) -int(dim2/10)
    
    mask = np.reshape(mask, (dim1, dim2)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage

In [12]:
def makeVideo(fileToProcess, destinationFolder):

    fileName = fileToProcess.split('/')[-1] #\\ if windows, / if on mac or sherlock
                                             #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()

    if not os.path.exists(os.path.join(destinationFolder,fileName[:-4]+'.avi')):
        # print(os.path.join(destinationFolder,fileName[:-4]+'.avi'))

        dataset = dicom.dcmread(fileToProcess, force=True)
        testarray = dataset.pixel_array
        if len(testarray.shape)!=4:
            return 0

#         frame0 = testarray[0]
#         plt.imshow(frame0[:,:,2])
#         mean = np.mean(frame0, axis=1)[:,0]
# #         mean = np.mean(mean, axis=1)
# #         plt.plot(mean)
#         try:
#             yCrop = np.where(mean<1)[0][0]
#             testarray = testarray[:, yCrop:, :, :]
#         except:
#             pass

        bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
        if bias>0:
            if testarray.shape[1] < testarray.shape[2]:
                testarray = testarray[:, :, bias:-bias, :]
            else:
                testarray = testarray[:, bias:-bias, :, :]

        # print(.shape)
        frames,height,width,channels = testarray.shape

        fps = 30

        try:
            fps = dataset[(0x18, 0x40)].value
        except:
            print("couldn't find frame rate, default to 30")

        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
        video_filename = os.path.join(destinationFolder, fileName[:-4] + '.avi')
        out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)


        for i in range(frames):

            outputA = testarray[i,:,:,0]
            outputA = mask(outputA)
            outputA = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

#             # Resize image
            finaloutput = cv2.resize(outputA, cropSize, interpolation = cv2.INTER_CUBIC)
            # finaloutput = mask(output)


            finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
            out.write(finaloutput)

        out.release()

    else:
        print(fileName,"hasAlreadyBeenProcessed")

In [13]:
## Convert to AVI 
cropSize = (112,112)
onlyfiles = [f for f in listdir(inputFolder) if isfile(join(inputFolder, f))]
# print(onlyfiles)
for f in tqdm.tqdm(onlyfiles[1:]):
    makeVideo(os.path.join(inputFolder,f),AVIFolder)


  0%|                                                   | 0/102 [00:00<?, ?it/s]

Dynamic-OHT21-055-2.dcm hasAlreadyBeenProcessed
Dynamic-OHT21-032.dcm hasAlreadyBeenProcessed
Dynamic-OHT21-068-2.dcm hasAlreadyBeenProcessed
Dynamic-OFF22-023-2.dcm hasAlreadyBeenProcessed
couldn't find frame rate, default to 30


 11%|████▌                                     | 11/102 [00:14<02:44,  1.81s/it]

couldn't find frame rate, default to 30


 16%|██████▌                                   | 16/102 [00:26<03:27,  2.42s/it]

...

 78%|████████████████████████████████▉         | 80/102 [03:14<00:43,  1.96s/it]

couldn't find frame rate, default to 30


 88%|█████████████████████████████████████     | 90/102 [03:35<00:18,  1.52s/it]

couldn't find frame rate, default to 30


100%|█████████████████████████████████████████| 102/102 [04:17<00:00,  2.52s/it]


In [14]:
# David Ouyang 12/5/2019

# Notebook which:
# 1. Downloads weights
# 2. Initializes model and imports weights
# 3. Performs test time evaluation of videos (already preprocessed with ConvertDICOMToAVI.ipynb)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import pathlib
import tqdm
import scipy 
import skimage

sys.path.append("..")
import echonet

import wget 

videosFolder = AVIFolder
#DestinationForWeights = "/Users/davidouyang/Dropbox/Echo Research/CodeBase/EchoNetDynamic-Weights"
DestinationForWeights = "/Users/vt2113/DonorAI/dynamic/Weights"

In [15]:
# Download model weights

if os.path.exists(DestinationForWeights):
    print("The weights are at", DestinationForWeights)
else:
    print("Creating folder at ", DestinationForWeights, " to store weights")
    os.mkdir(DestinationForWeights)
    
segmentationWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/deeplabv3_resnet50_random.pt'
ejectionFractionWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'


if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL))):
    print("Downloading Segmentation Weights, ", segmentationWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(segmentationWeightsURL)))
    filename = wget.download(segmentationWeightsURL, out = DestinationForWeights)
else:
    print("Segmentation Weights already present")
    
if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL))):
    print("Downloading EF Weights, ", ejectionFractionWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(ejectionFractionWeightsURL)))
    filename = wget.download(ejectionFractionWeightsURL, out = DestinationForWeights)
else:
    print("EF Weights already present")
        


The weights are at /Users/vt2113/DonorAI/dynamic/Weights
Segmentation Weights already present
EF Weights already present


In [16]:
# Set up model
model_name = 'deeplabv3_resnet50'
pretrained = False
model = torchvision.models.segmentation.__dict__[model_name](pretrained=pretrained, aux_loss=False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size=model.classifier[-1].kernel_size)  # change number of outputs to 1

print("loading weights from ", os.path.join(DestinationForWeights, "deeplabv3_resnet50_random"))


if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)

loading weights from  /Users/vt2113/DonorAI/dynamic/Weights/deeplabv3_resnet50_random
cuda is not available, cpu weights


In [17]:
# Saving videos with segmentations
ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder,target_type=["Filename"])
print(ds.split, ds.fnames)
output = destinationFolder
save_video = True

mean, std = echonet.utils.get_mean_and_std(ds)
num_workers = 0
batch_size = 64
def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i

dataset = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder,target_type=["Filename"],  # Need filename for saving, and human-selected frames to annotate
                                mean=mean, std=std,  # Normalization
                                length=None, max_length=None, period=1  # Take all frames
                                )
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=num_workers, shuffle=False, pin_memory=False, collate_fn=collate_fn)

# Save videos with segmentation
if save_video and not all(os.path.isfile(os.path.join(output, "videos", f)) for f in dataloader.dataset.fnames):
    # Only run if missing videos

    model.eval()

    os.makedirs(os.path.join(output, "videos"), exist_ok=True)
    os.makedirs(os.path.join(output, "size"), exist_ok=True)
    echonet.utils.latexify()

    with torch.no_grad():
        with open(os.path.join(output, "size.csv"), "w") as g:
            g.write("Filename,Frame,Size,ComputerSmall\n")
            for (x, (filenames), length) in tqdm.tqdm(dataloader):
                # Run segmentation model on blocks of frames one-by-one
                # The whole concatenated video may be too long to run together
                y = np.concatenate([model(x[i:(i + batch_size), :, :, :].to(device))["out"].detach().cpu().numpy() for i in range(0, x.shape[0], batch_size)])

                start = 0
                x = x.numpy()

                for (i, (filename, offset)) in enumerate(zip(filenames, length)):
                    # Extract one video and segmentation predictions
                    video = x[start:(start + offset), ...]
                    logit = y[start:(start + offset), 0, :, :]

                    # Un-normalize video
                    video *= std.reshape(1, 3, 1, 1)
                    video += mean.reshape(1, 3, 1, 1)

                    # Get frames, channels, height, and width
                    f, c, h, w = video.shape  # pylint: disable=W0612
                    assert c == 3

                    # Put two copies of the video side by side
                    video = np.concatenate((video, video), 3)

                    # If a pixel is in the segmentation, saturate blue channel
                    # Leave alone otherwise
                    video[:, 0, :, w:] = np.maximum(255. * (logit > 0), video[:, 0, :, w:])  # pylint: disable=E1111

                    # Add blank canvas under pair of videos
                    video = np.concatenate((video, np.zeros_like(video)), 2)

                    # Compute size of segmentation per frame
                    size = (logit > 0).sum((1, 2))

                    # Identify systole frames with peak detection
                    trim_min = sorted(size)[round(len(size) ** 0.05)]
                    trim_max = sorted(size)[round(len(size) ** 0.95)]
                    trim_range = trim_max - trim_min
                    systole = set(scipy.signal.find_peaks(-size, distance=20, prominence=(0.50 * trim_range))[0])

                    # Write sizes and frames to file
                    for (frame, s) in enumerate(size):
                        g.write("{},{},{},{}\n".format(filename, frame, s,1 if frame in systole else 0))

                    # Plot sizes
                    fig = plt.figure(figsize=(size.shape[0] / 50 * 1.5, 3))
                    plt.scatter(np.arange(size.shape[0]) / 50, size, s=1)
                    ylim = plt.ylim()
                    for s in systole:
                        plt.plot(np.array([s, s]) / 50, ylim, linewidth=1)
                    plt.ylim(ylim)
                    plt.title(os.path.splitext(filename)[0])
                    plt.xlabel("Seconds")
                    plt.ylabel("Size (pixels)")
                    plt.tight_layout()
                    plt.savefig(os.path.join(output, "size", os.path.splitext(filename)[0] + ".pdf"))
                    plt.close(fig)

                    # Normalize size to [0, 1]
                    size -= size.min()
                    size = size / size.max()
                    size = 1 - size

                    # Iterate the frames in this video
                    for (f, s) in enumerate(size):

                        # On all frames, mark a pixel for the size of the frame
                        try:
                            video[:, :, int(round(115 + 100 * s)), int(round(f / len(size) * 200 + 10))] = 255.
                        except:
                            pass

                        if f in systole:
                            # If frame is computer-selected systole, mark with a line

                            video[:, :, 115:224, int(round(f / len(size) * 200 + 10))] = 255.

                        def dash(start, stop, on=10, off=10):
                            buf = []
                            x = start
                            while x < stop:
                                buf.extend(range(x, x + on))
                                x += on
                                x += off
                            buf = np.array(buf)
                            buf = buf[buf < stop]
                            return buf
                        d = dash(115, 224)

                        try:
                            # Get pixels for a circle centered on the pixel
                            r, c = skimage.draw.disk((int(round(115 + 100 * s)), int(round(f / len(size) * 200 + 10))), 4.1)

                            # On the frame that's being shown, put a circle over the pixel
                            video[f, :, r, c] = 255.
                        except:
                            pass

                    # Rearrange dimensions and save
                    video = video.transpose(1, 0, 2, 3)
                    video = video.astype(np.uint8)
                    echonet.utils.savevideo(os.path.join(output, "videos", filename), video, 50)

                    # Move to next video
                    start += offset

  0%|                                                    | 0/13 [00:00<?, ?it/s]

EXTERNAL_TEST ['Dynamic-OFF22-021.avi', 'Dynamic-OFF22-022.avi', 'Dynamic-OFF22-023-1.avi', 'Dynamic-OFF22-023-2.avi', 'Dynamic-OFF22-024.avi', 'Dynamic-OFF22-025.avi', 'Dynamic-OFF22-026.avi', 'Dynamic-OFF22-027.avi', 'Dynamic-OFF22-028.avi', 'Dynamic-OFF22-029.avi', 'Dynamic-OFF22-030.avi', 'Dynamic-OFF22-031.avi', 'Dynamic-OFF22-032..avi', 'Dynamic-OFF22-033.avi', 'Dynamic-OFF22-034.avi', 'Dynamic-OFF22-035.avi', 'Dynamic-OFF22-036.avi', 'Dynamic-OFF22-037.avi', 'Dynamic-OFF22-038.avi', 'Dynamic-OFF22-039.avi', 'Dynamic-OFF22-040.avi', 'Dynamic-OFF22-041.avi', 'Dynamic-OFF22-042.avi', 'Dynamic-OFF22-043.avi', 'Dynamic-OFF22-045.avi', 'Dynamic-OFF22-046.avi', 'Dynamic-OFF22-047.avi', 'Dynamic-OFF22-048.avi', 'Dynamic-OFF22-049.avi', 'Dynamic-OFF22-050.avi', 'Dynamic-OFF22-051.avi', 'Dynamic-OFF22-052.avi', 'Dynamic-OFF22-053.avi', 'Dynamic-OFF22-055.avi', 'Dynamic-OHT21-003.avi', 'Dynamic-OHT21-005.avi', 'Dynamic-OHT21-006.avi', 'Dynamic-OHT21-007.avi', 'Dynamic-OHT21-011.avi', 'Dyna

 21%|████████▊                                  | 20/97 [07:08<22:21, 17.42s/it]/var/folders/hf/_4l7f46s7sx36_h8zwshkn_41s8kcx/T/ipykernel_34426/3205764825.py:89: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()
100%|███████████████████████████████████████████| 97/97 [32:46<00:00, 20.27s/it]


In [18]:
# Initialize and Run EF model

frames = 32
period = 2 #2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)



print("loading weights from ", os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained"))

if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)


# try some random weights: final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar
# scp ouyangd@arthur2:~/Echo-Tracing-Analysis/final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar "C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights"
#Weights = "final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar"


output = os.path.join(destinationFolder, "ef_output.csv")

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder)
print(ds.split, ds.fnames)

mean, std = echonet.utils.get_mean_and_std(ds)

kwargs = {"target_type": "EF",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period,
          }

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, **kwargs)

test_dataloader = torch.utils.data.DataLoader(ds, batch_size = 1, num_workers = 5, shuffle = True, pin_memory=(device.type == "cuda"))
loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader,False, None, device, save_all=True)

with open(output, "w") as g:
    for (filename, pred) in zip(ds.fnames, yhat):
        for (i,p) in enumerate(pred):
            g.write("{},{},{:.4f}\n".format(filename, i, p))

loading weights from  /Users/vt2113/DonorAI/dynamic/Weights/r2plus1d_18_32_2_pretrained
cuda is not available, cpu weights


  0%|                                                    | 0/13 [00:00<?, ?it/s]

EXTERNAL_TEST ['Dynamic-OFF22-021.avi', 'Dynamic-OFF22-022.avi', 'Dynamic-OFF22-023-1.avi', 'Dynamic-OFF22-023-2.avi', 'Dynamic-OFF22-024.avi', 'Dynamic-OFF22-025.avi', 'Dynamic-OFF22-026.avi', 'Dynamic-OFF22-027.avi', 'Dynamic-OFF22-028.avi', 'Dynamic-OFF22-029.avi', 'Dynamic-OFF22-030.avi', 'Dynamic-OFF22-031.avi', 'Dynamic-OFF22-032..avi', 'Dynamic-OFF22-033.avi', 'Dynamic-OFF22-034.avi', 'Dynamic-OFF22-035.avi', 'Dynamic-OFF22-036.avi', 'Dynamic-OFF22-037.avi', 'Dynamic-OFF22-038.avi', 'Dynamic-OFF22-039.avi', 'Dynamic-OFF22-040.avi', 'Dynamic-OFF22-041.avi', 'Dynamic-OFF22-042.avi', 'Dynamic-OFF22-043.avi', 'Dynamic-OFF22-045.avi', 'Dynamic-OFF22-046.avi', 'Dynamic-OFF22-047.avi', 'Dynamic-OFF22-048.avi', 'Dynamic-OFF22-049.avi', 'Dynamic-OFF22-050.avi', 'Dynamic-OFF22-051.avi', 'Dynamic-OFF22-052.avi', 'Dynamic-OFF22-053.avi', 'Dynamic-OFF22-055.avi', 'Dynamic-OHT21-003.avi', 'Dynamic-OHT21-005.avi', 'Dynamic-OHT21-006.avi', 'Dynamic-OHT21-007.avi', 'Dynamic-OHT21-011.avi', 'Dyna

100%|███████████████████████████████████████████| 13/13 [00:30<00:00,  2.31s/it]
/Users/vt2113/opt/anaconda3/envs/echonet/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████| 97/97 [06:56<00:00,  4.29s/it, 3297.50 (4060.33) / 0.00]


In [19]:
def extract_pixel(inputFolder,destination):
    onlyfiles = [f for f in listdir(inputFolder) if (isfile(join(inputFolder, f)) and f[-4:] ==".dcm")]
    fnames = [f.split('/')[-1] for f in onlyfiles]
    x_dim = []
    y_dim = []
    x = []
    y = []
    original_shape = []
    for f in tqdm.tqdm(onlyfiles):
        dataset = dicom.dcmread(os.path.join(inputFolder,f), force=True)
        x_dim.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x6024)].value)
        y_dim.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x6026)].value)
        x.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x602c)].value)
        y.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x602e)].value)
        original_shape.append(dataset.pixel_array.shape)
        
        
    df = pd.DataFrame({
        "fname":fnames,
        "x_unit":x_dim,
        "y_unit":y_dim,
        "delta_x":x,
        "delta_y":y,
        "Original_dim":original_shape
    })
    df.to_csv(destination+os.path.sep+'meta_Data.csv')

In [20]:
## Convert to AVI 
cropSize = (112,112)
onlyfiles = [f for f in listdir(inputFolder) if isfile(join(inputFolder, f))]
# for f in onlyfiles:
    # makeVideo(os.path.join(inputFolder,f),AVIFolder)    

extract_pixel(inputFolder,destinationFolder)


100%|█████████████████████████████████████████| 103/103 [04:02<00:00,  2.35s/it]


In [21]:
def func(size,file,scale):
    x,y = scale[file]
    return size*x*y
    
    
    
def convert_px_to_cm(input_folder):
    resize = (112,112)
    # onlyfiles = [f for f in listdir(input_folder) if not isfile(join(input_folder,f))]
    df_meta = pd.read_csv(os.path.join(input_folder,'meta_Data.csv'))
    df_size = pd.read_csv(os.path.join(input_folder,'size.csv'))
    onlyfiles = df_size.Filename.unique()
    scale_dict = {}
    for f in tqdm.tqdm(onlyfiles):
        df_now = df_meta[df_meta.fname==f[:-4]+".dcm"]
        dims = eval(list(df_now.Original_dim)[0])
        x_scale = abs(df_now.delta_x * 0.8*dims[1]/resize[0]).to_numpy()[0]
        y_scale = abs(df_now.delta_y * 0.8*dims[1]/resize[1]).to_numpy()[0]
        scale_dict[f] = [x_scale,y_scale]
        

        
    df_size['size_cm^2'] = df_size.apply(lambda x: func(x.Size, x.Filename,scale_dict), axis=1)
    df_size.to_csv(os.path.join(input_folder,'size_cm.csv'))

 
    
    
    

        
        
        

In [22]:
convert_px_to_cm(destinationFolder)

100%|██████████████████████████████████████████| 97/97 [00:00<00:00, 265.90it/s]
